# Анализ старых игр для команды «Секреты Темнолесья»
- Автор: Котова Ольга 
- Дата: 02.02.2025

### Цели и задачи проекта
<font color='#777778'>Цель анализа: написание статьи  о развитии индустрии игр в начале XXI века для привлечения новой аудитории к "Секретам Темнолесья".
Необходимо: 
- провести обзор игровых платформ, 
- изучить объёмы продаж игр разных жанров и региональные предпочтения игроков. 
Период изучения: с 2000 по 2013 год.

**Акцент на играх жанра RPG**.

Источник данных: исторические данные собраны из открытых источников "Секретами Темнолесья":
- о продажах игр, сделанных в разных жанрах и выпущенных на разных платформах, 
- пользовательские и экспертные оценки.

Задачи:

- познакомиться с данными, 
- проверить их корректность и провести предобработку,
- получить необходимый срез данных.

### Описание данных

`Name` — название игры.  
`Platform` — название платформы.  
`Year of Release` — год выпуска игры.  
`Genre` — жанр игры.  
`NA sales` — продажи в Северной Америке (в миллионах проданных копий).  
`EU sales` — продажи в Европе (в миллионах проданных копий).  
`JP sales` — продажи в Японии (в миллионах проданных копий).  
`Other sales` — продажи в других странах (в миллионах проданных копий).  
`Critic Score` — оценка критиков (от 0 до 100).  
`User Score` — оценка пользователей (от 0 до 10).  
`Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board) c подходящей возрастной категорией.  

### Содержимое проекта
1. Загрузка данных и знакомство с ними
2. Проверка ошибок в данных и их предобработка  
    2.1. Названия, или метки, столбцов датафрейма  
    2.2. Типы данных  
    2.3. Наличие пропусков в данных  
    2.4. Явные и неявные дубликаты в данных  
3. Фильтрация данных  
4. Категоризация данных
5. Выводы

## 1. Загрузка данных и знакомство с ними
Загружаем необходимые библиотеки Python и данные датасета /datasets/new_games.csv.
Знакомимся с данными: вывели первые строки и результат метода info().


In [6]:
# Импортируем библиотеку pandas
import pandas as pd
pd.options.mode.chained_assignment = None
# Выгружаем данные в датафрейм
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')


df.info()

df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


**Вывод о полученных данных**: 
нам предоставлены данные  в 16956 строк и 11 столбцов. Они соответствуют описанию.

В данных встречаются пропуски: в столбцах в Name и Genre по 2 пропуска,
пропусков больше в Year of Release, Critic Score,  User Score,  Rating.
В дальнейшем следует оценить объем пропусков стоит через их долю.

Верный тип данных object  в Name, Platform, Genre, Rating.    
В Year of Release  тип данных не временной - float64, что неверно, его следует заменить, но поскольку нас инетерсует только год, а не конкретные даты и время в анализе, приведем к целочисленному значению.  
Во всех продажах небходим числовой тип данных, float64 только у NA sales, Other sales, треубет замены EU sales, JP sales.  
Столбец Critic Score также в числовом формате, но следует оценить, необходим ли float64 или разрядность можно уменьшить.  
Столбец User Score следует пересевсти из object в числовой формат.

В целом названия столбцов отражают содержание данных, однако для приведению к стилю snake case следует дать названия в нижнем регистре и с нижним подчеркиванием вместо пробелов.

## 2.  Проверка ошибок в данных и их предобработка


### 2.1. Названия, или метки, столбцов датафрейма

- Выводим на экран названия всех столбцов датафрейма и проверяем их стиль написания.

In [9]:
#Во избежании удаления лишних данных в исходном датафрейме, создадим его копию и будем работать в ней
games=pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')
games.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

- Приводим все столбцы к стилю snake case

In [11]:
 def columns_snake_case (column):
     return column.lower().replace(' ', '_')

games.columns = [columns_snake_case(column) for column in games.columns]

games.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### 2.2. Типы данных

-  Некорректные типы данных и их причины:  
Год year_of_release выражен числом, в этом может быть причина присвоения float64, нам следует его преобразовать в `int32`.  
Во всех продажах небходим числовой тип данных, треубет замены eu_sales, jp_sales, other_sales, вероятно, в столбцах встречались строковые значения, поэтому всему столбцу присвоен строковый тип. 
Столбец critic_score во `float64б` попробуем разрядность снизить, хотя числовой формат - это верно  
Столбец user_score следует пересевсти из `object` в числовой формат, причина неверного формата опять
может скрываться в наличии строковых значений
- Проведем необходимые преобразование типов данных.
Помним, что столбцы с числовыми данными и пропусками нельзя преобразовать к типу `int64`. Сначала понадобится обработать пропуски, а затем преобразовать типы данных.
- В числовых столбцах могут встретиться строковые значения, например `unknown` или другие. Приводим такие столбцы к числовому типу данных, заменив строковые значения на пропуски - метод to_numeric строковые значения, такие как `unknown`, заменит на NaN, а числовые значения - переведет вок типу float.
- Также можно разделить данные на категории, например, в рейтинге.

In [13]:
#Сначала заменяем пропуски на 0, без этого код выдаст ошибку
games['year_of_release'] = games['year_of_release'].fillna(0)
#Меняем тип на целое число, выбираем разрядость поменьше для снижения загруженности памяти - int32
games['year_of_release'] = games['year_of_release'].astype('int32')
games['year_of_release']

0        2006
1        1985
2        2008
3        2009
4        1996
         ... 
16951    2016
16952    2006
16953    2016
16954    2003
16955    2016
Name: year_of_release, Length: 16956, dtype: int32

In [14]:
#меняем тип данных в продажах и оценках на оптимальный, при этом строковые значения будут заменены на NaN, 
#а числовые значения были приведены к типу float, по столбцу critic_score - уменьшена разрядность.
for column in ['eu_sales', 'jp_sales', 'other_sales', 'critic_score', 'user_score']:
    games [column]= pd.to_numeric (games [column], errors = 'coerce', downcast = 'float')

#меняем тип данных в регйтинге на категориальный
games['rating'] = games['rating'].astype('category')
games.dtypes

name                 object
platform             object
year_of_release       int32
genre                object
na_sales            float64
eu_sales            float32
jp_sales            float32
other_sales         float32
critic_score        float32
user_score          float32
rating             category
dtype: object

### 2.3. Наличие пропусков в данных

- Считаем количество пропусков в каждом столбце в абсолютных и относительных значениях.


In [16]:
#выводим количество пропусков в каждом столбце в абсолютном значении
games.isna().sum()

name                  2
platform              0
year_of_release       0
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [17]:
#выводим количество пропусков в каждом столбце в относительном значении
games.isna().mean()

name               0.000118
platform           0.000000
year_of_release    0.000000
genre              0.000118
na_sales           0.000000
eu_sales           0.000354
jp_sales           0.000236
other_sales        0.000000
critic_score       0.513918
user_score         0.546591
rating             0.405225
dtype: float64

**Промежуточный вывод**
- По 2 пропуска в name и genre - их процент незначителен, можно их удалить.
- Пропуски в sales также незначительны, они нам в целом не требуются для анализа на данном этапе, в всязи с чем их можно пригнорировать.   
А можно количество проданных копий игры в том или ином регионе, заменить на среднее значение в зависимости от названия платформы и года выхода игры.
- Заметное число пропусков следующих столбцах, их можно заменить на индикатор "отсутствует": 
critic_score    -   8714,
user_score      -   9268,
rating           -  6871.
Доля пропусков высока. Вероятно, пропуски здесь возникли в связи с тем, что анализ проводится по старым годам, когда оценки было учесть не так легко, как в наши дни, тыкнув на звездочку в приложении, а данные собраны из открытых источников.

In [19]:
#удаляем пропуски в name и genre 
games.dropna(subset=['name', 'genre'], inplace=True)
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16954 non-null  object  
 1   platform         16954 non-null  object  
 2   year_of_release  16954 non-null  int32   
 3   genre            16954 non-null  object  
 4   na_sales         16954 non-null  float64 
 5   eu_sales         16948 non-null  float32 
 6   jp_sales         16950 non-null  float32 
 7   other_sales      16954 non-null  float32 
 8   critic_score     8242 non-null   float32 
 9   user_score       7688 non-null   float32 
 10  rating           10085 non-null  category
dtypes: category(1), float32(5), float64(1), int32(1), object(3)
memory usage: 1.1+ MB


In [20]:
# В столбцах eu_sales, jp_sales меняем пропуски на среднее значение в зависимости от названия платформы и года выхода игры

# вводим переменную индекса для столбцов продаж, чтобы функция присвоения среднего значения работала по каждому из 3 столбцов с пропусками
col_sales_index=0 
# создаем пользовательскую функцию лоя расчета среднего значения
def mean_group_sales(row): 
    columns_sales=['eu_sales', 'jp_sales']
    if pd.isna(row[columns_sales[col_sales_index]]): 
        group = games[(games['platform'] == row['platform']) & (games['year_of_release'] == row['year_of_release'])]
        return group[columns_sales[col_sales_index]].mean()  # Возврат среднего значения
    else:
        return row[columns_sales[col_sales_index]]  # Возврат текущего значения, если оно не NaN
        
# в цикле перебираем необходимые столбцы, в которых надо заменить пропуски средним значением
for i in ['eu_sales', 'jp_sales']:
# Применяем функцию к каждой строке
    games[i] = games.apply(mean_group_sales, axis=1)
# увеличиваем переменную индекса для столбцов продаж, чтобы применить функцию к следующему столбцу из списка   
    col_sales_index+=1 
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16954 non-null  object  
 1   platform         16954 non-null  object  
 2   year_of_release  16954 non-null  int32   
 3   genre            16954 non-null  object  
 4   na_sales         16954 non-null  float64 
 5   eu_sales         16954 non-null  float64 
 6   jp_sales         16954 non-null  float64 
 7   other_sales      16954 non-null  float32 
 8   critic_score     8242 non-null   float32 
 9   user_score       7688 non-null   float32 
 10  rating           10085 non-null  category
dtypes: category(1), float32(3), float64(3), int32(1), object(3)
memory usage: 1.2+ MB


In [21]:
# в столбцах со значительным количеством пропусков меняем пропуски в оценках на индикаторы: для оценок -1
games['critic_score'] = games['critic_score'].fillna(-1) 
games['user_score'] = games['user_score'].fillna(-1)  
# заполнение пропусков в колонке 'rating' возможно только через добавление новой категории, иначе код выдаст ошибку
games['rating'] = games['rating'].cat.add_categories('no rating')  
games['rating'] = games['rating'].fillna('no rating')  
games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16954 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   name             16954 non-null  object  
 1   platform         16954 non-null  object  
 2   year_of_release  16954 non-null  int32   
 3   genre            16954 non-null  object  
 4   na_sales         16954 non-null  float64 
 5   eu_sales         16954 non-null  float64 
 6   jp_sales         16954 non-null  float64 
 7   other_sales      16954 non-null  float32 
 8   critic_score     16954 non-null  float32 
 9   user_score       16954 non-null  float32 
 10  rating           16954 non-null  category
dtypes: category(1), float32(3), float64(3), int32(1), object(3)
memory usage: 1.2+ MB


В итоговом выводе через info() видим, что все пропуски удалены - по всем 16954 строкам 16954 non-null значений.

### 2.4. Явные и неявные дубликаты в данных

- Изучим уникальные значения в категориальных данных, например с названиями жанра игры, платформы, рейтинга и года выпуска, на наличие среди данных неявных дубликатов, связанных с опечатками или разным способом написания.


In [24]:
# выводим уникальные значения по столбцам названиями жанра игры, платформы, рейтинга и года выпуска
for category_colomn in ['genre', 'platform', 'rating', 'year_of_release']:
    display(f'Уникальные значения столбца {category_colomn}:')
    display(games[category_colomn].unique())

'Уникальные значения столбца genre:'

array(['Sports', 'Platform', 'Racing', 'Role-Playing', 'Puzzle', 'Misc',
       'Shooter', 'Simulation', 'Action', 'Fighting', 'Adventure',
       'Strategy', 'MISC', 'ROLE-PLAYING', 'RACING', 'ACTION', 'SHOOTER',
       'FIGHTING', 'SPORTS', 'PLATFORM', 'ADVENTURE', 'SIMULATION',
       'PUZZLE', 'STRATEGY'], dtype=object)

'Уникальные значения столбца platform:'

array(['Wii', 'NES', 'GB', 'DS', 'X360', 'PS3', 'PS2', 'SNES', 'GBA',
       'PS4', '3DS', 'N64', 'PS', 'XB', 'PC', '2600', 'PSP', 'XOne',
       'WiiU', 'GC', 'GEN', 'DC', 'PSV', 'SAT', 'SCD', 'WS', 'NG', 'TG16',
       '3DO', 'GG', 'PCFX'], dtype=object)

'Уникальные значения столбца rating:'

['E', 'no rating', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP']
Categories (9, object): ['AO', 'E', 'E10+', 'EC', ..., 'M', 'RP', 'T', 'no rating']

'Уникальные значения столбца year_of_release:'

array([2006, 1985, 2008, 2009, 1996, 1989, 1984, 2005, 1999, 2007, 2010,
       2013, 2004, 1990, 1988, 2002, 2001, 2011, 1998, 2015, 2012, 2014,
       1992, 1997, 1993, 1994, 1982, 2016, 2003, 1986, 2000,    0, 1995,
       1991, 1981, 1987, 1980, 1983])

Видим наличие неявных дубликатов, типа 'Sports' и 'SPORTS' в жанре.  
Проведем нормализацию данных с текстовыми значениями. Жанры игр приведем к нижнему регистру, а названия рейтинга — к верхнему.

In [26]:
# В целях минимизации неявных дубликатов, таких как 'Sports' и 'SPORTS', приводим данные в столбцe genre в нижний регистр
games['genre']=games['genre'].str.lower()
display(games['genre'].unique())


array(['sports', 'platform', 'racing', 'role-playing', 'puzzle', 'misc',
       'shooter', 'simulation', 'action', 'fighting', 'adventure',
       'strategy'], dtype=object)

In [27]:
games['rating']=games['rating'].str.upper()
display(games['rating'].unique())

array(['E', 'NO RATING', 'M', 'T', 'E10+', 'K-A', 'AO', 'EC', 'RP'],
      dtype=object)

- Проверим наличие явных дубликатов в данных.

In [29]:
#ищем количество явных дубликатов
games.duplicated().sum()


241

Найден 241 явный дубликат, в целях качественного анализа удаляем их, сохраняя первую запись в новом датафрейме games_clean

In [31]:
# удаляем дубликаты, сохраняя первую запись в новом датафрейме
games_clean=games.drop_duplicates(subset=None, keep='first', inplace=False)
games_clean. info()

<class 'pandas.core.frame.DataFrame'>
Index: 16713 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16713 non-null  object 
 1   platform         16713 non-null  object 
 2   year_of_release  16713 non-null  int32  
 3   genre            16713 non-null  object 
 4   na_sales         16713 non-null  float64
 5   eu_sales         16713 non-null  float64
 6   jp_sales         16713 non-null  float64
 7   other_sales      16713 non-null  float32
 8   critic_score     16713 non-null  float32
 9   user_score       16713 non-null  float32
 10  rating           16713 non-null  object 
dtypes: float32(3), float64(3), int32(1), object(4)
memory usage: 1.3+ MB


- В процессе подготовки данных мы удаляли строки с пропусками и дубликатами, посчитаем количество удалённых строк в абсолютном и относительном значениях.

In [33]:
# Считаем количеcтво строк в загруженном изначально начальном датафрейме
start_row_count=df.shape[0]
# Считаем количеcтво строк в датафрейме после предобработки
final_row_count=games_clean.shape[0]
# Считаем разницу в строках начального и конечного датафреймов
difference_row=start_row_count-final_row_count
# Считаем долю удаленных строк от изначального количества строк в процентах
difference_row_share=round(difference_row/start_row_count*100,2)
display (f'Начальное число строк: {start_row_count}, в результате предобрабоки осталось: {final_row_count} строк. Удалено {difference_row} строк, или {difference_row_share} %')

'Начальное число строк: 16956, в результате предобрабоки осталось: 16713 строк. Удалено 243 строк, или 1.43 %'

**Oбщий промежуточный вывод**  
В процессе подготовки данных:
- исследовали объем и типы данных, скорректировали тип данных, оставив категоризацию на 4 этап проекта;  
- выявили наличие пропусков, которые заменили на 0 в годе выпуска, на среднее значение в продажах, удалили (при невесомом проценте пропусков), заменили на индикаторы в оценках и рейтинге;
- поработали с дубликатами: избавились от неявных дубликатов и удалили явные, в итоге осталось 16713 строк вместо исходных 16955. Удалено 243 строк, или 1.43 %


## 3. Фильтрация данных

Стояла задача изучить историю продаж игр в начале XXI века, а именно период с 2000 по 2013 год включительно. Отбираем данные по этому показателю и сохраняем новый срез данных в отдельном датафрейме - `df_actual`.

In [36]:
#фильтруем данные по годам
df_actual=games_clean[(games_clean['year_of_release'] >= 2000) & (games_clean['year_of_release'] <= 2013)]
#выведем уникальные годы нового датафрейма с фильтром по годам
df_actual['year_of_release'].unique()

array([2006, 2008, 2009, 2005, 2007, 2010, 2013, 2004, 2002, 2001, 2011,
       2012, 2003, 2000])


## 4. Категоризация данных
    
Проведем категоризацию данных:
- Разделим все игры по оценкам пользователей на такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [38]:
#создаем пользовательскую функцию для категоризации оценки пользователей
def categorize_user_score(user_score):
    if 8 <= user_score <= 10:
        return 'Высокая оценка'
    elif 3 <= user_score < 8:
        return 'Средняя оценка'
    elif 0 <= user_score < 3:
        return 'Низкая оценка'
    else:
        return 'нет данных'  # На случай, если рейтинг не в ожидаемых границах

# Применяем пользовательскую функцию к датафрейму и содаем новый столбец 'category_user_score'
df_actual['category_user_score'] = df_actual['user_score'].apply(categorize_user_score)
df_actual.head()


,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,category_user_score
0,Wii Sports,Wii,2006,sports,41.36,28.959999,3.77,8.45,76.0,8.0,E,Высокая оценка
2,Mario Kart Wii,Wii,2008,racing,15.68,12.760000,3.79,3.29,82.0,8.3,E,Высокая оценка
3,Wii Sports Resort,Wii,2009,sports,15.61,10.930000,3.28,2.95,80.0,8.0,E,Высокая оценка
6,New Super Mario Bros.,DS,2006,platform,11.28,9.140000,6.50,2.88,89.0,8.5,E,Высокая оценка
7,Wii Play,Wii,2006,misc,13.96,9.180000,2.93,2.84,58.0,6.6,E,Средняя оценка


- После категоризации данных проверим результат: сгруппируем данные по выделенным категориям и посчитаем количество игр в каждой категории.

In [40]:
# Группируем по категории оценки пользователей и считаем количество игр в каждой категории
category_user_counts = df_actual.groupby('category_user_score')['name'].count()
# Проверяем результат
display(category_user_counts)

category_user_score
Высокая оценка    2286
Низкая оценка      116
Средняя оценка    4081
нет данных        6298
Name: name, dtype: int64

- Выделим топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [42]:
# сортируем по количеству игр 
category_platform_counts = df_actual['platform'].value_counts()
# выводим первые 7 строк, чтобы получить топ-7
category_platform_counts.head(7)

platform
PS2     2127
DS      2120
Wii     1275
PSP     1180
X360    1121
PS3     1087
GBA      811
Name: count, dtype: int64


## 5. Итоговый вывод

- Были загружены данные `datasets/new_games.csv`, содержащие 16956 строк и 11 столбцов.  
- Названия всех столбцов к стилю snake case.  
- Для оптимизации работы с данными в датафрейме были произведены изменения типов данных: в столбце 'year_of_release' изменили тип данных с float64 на  целочисленный int32 (с заменой пропусков на 0),  в продажах 'eu_sales', 'jp_sales', 'other_sales' и оценках 'critic_score', 'user_score' на оптимальный float32, по столбцу 'critic_score' - уменьшена разрядность. В рейтинге установлен категориальный тип данных.
- В 6 столбцах 'name', 'genre', 'eu_sales', 'jp_sales', 'critic_score', 'user_score' были обнаружены пропущенные значения. По первым двум 2 столбцам'name', 'genre' данные были удалены (всего 2 пропуска), в столбцах 'eu_sales', 'jp_sales'- пропуски заполнены средними значениями, в 'critic_score', 'user_score', 'rating' со значительным количеством пропусков заменили пропуски на индикаторы: для оценок -1, для рейтинга - на 'no rating'.
- Обнаружены неявные дубликаты, проведена работы по борьбе с ними, также найден и удален 241 явный дубликат.
- Получен срез датафрейма за период с 2000 по 2013 год, добавлено новое поле 'category_user_score' с рейтингом по оценкам пользователей: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала). По результатам катгоризации по количеству игр получили следующую градацию:
Высокая оценка    2286
Низкая оценка      116  
Средняя оценка    4081  
нет данных        6298  

- Выделено топ-7 платформ по количеству игр, выпущенных за весь актуальный период: PS2, DS, Wii, PSP, X360, PS3, GBA.  